In [ ]:
import pandas as pd


In [1]:
!dir inputs

 ドライブ C のボリューム ラベルがありません。
 ボリューム シリアル番号は 5AD7-2869 です

 C:\Users\yanai\OneDrive\ドキュメント\Python Scripts\Signate\【練習問題】林型の分類\inputs のディレクトリ

11/03/2021  19:26    <DIR>          .
11/03/2021  19:26    <DIR>          ..
11/03/2021  18:56         2,559,052 sample_submit.csv
11/03/2021  18:56        38,982,818 test.tsv
11/03/2021  18:56        39,562,946 train.tsv
               3 個のファイル          81,104,816 バイト
               2 個のディレクトリ  173,260,460,032 バイトの空き領域


In [ ]:
train = pd.read_csv('inputs/train.tsv', sep='\t', index_col=0)
train

In [2]:
train['Cover_Type'].hist()

NameError: name 'train' is not defined

In [ ]:
train['Cover_Type'].value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
X = train.drop('Cover_Type', axis=1)
y = train['Cover_Type']

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
y = y-1

In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
skf = StratifiedKFold(n_splits=10,
                      shuffle=True,
                      random_state=0)

In [ ]:
# LightGBM parameters
params = {
        'task': 'train',
#         'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': {'multi_logloss'},
        'num_class': 7,
#         'learning_rate': 0.1,
#        'num_leaves': 23,
 #       'min_data_in_leaf': 1,
#         'num_iteration': 100,
        'verbose': 0
}

In [ ]:
# LightGBM parameters
param_grid = {
#     "max_depth": [10, 25, 50, 75],
              "learning_rate" : [0.01, 0.1],
#               "num_leaves": [100,300],
              "n_estimators": [100, 300, 900]
             }
# param_grid = {"max_depth": [10, 25, 50, 75],
#               "learning_rate" : [0.001,0.01,0.1],
#               "num_leaves": [100,300],
#               "n_estimators": [100,200]
#              }
# param_grid = {"max_depth": [10, 25, 50, 75],
#               "learning_rate" : [0.001,0.01,0.05,0.1],
#               "num_leaves": [100,300,900,1200],
#               "n_estimators": [100,200,500]
#              }

In [ ]:
model = lgb.LGBMClassifier()

In [ ]:
# パラメータチューニングをグリッドサーチ
grid_result = GridSearchCV(estimator = model,
                           param_grid = param_grid,
                           scoring = 'balanced_accuracy', #accuracy, balanced_accuracy
                           cv = skf,
                           return_train_score = True,
                           n_jobs = -1)

In [ ]:
grid_result.fit(X_train, y_train)

In [ ]:
# # train
# gbm = lgb.train(params,
#             lgb_train,
#             num_boost_round=50,
#             valid_sets=lgb_eval,
#             early_stopping_rounds=10)


In [ ]:
grid_result.best_params_

In [ ]:
y_pred = grid_result.predict(X_test)
# y_pred = np.argmax(y_pred, axis=1)
y_pred

In [ ]:
np.unique(y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

0.8746686861037486<br/>
{'learning_rate': 0.1, 'n_estimators': 200}